In [4]:
!python --version
!pip install chess

import numpy as np
from tqdm.notebook import trange
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot

import torch
import torch.nn as nn
import torch.functional as F
from torch.utils.tensorboard import SummaryWriter

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

Python 3.7.15
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cuda:0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import chess
import chess.pgn

class State:
  def __init__(self, board=None):
    if board is None:
      self.board = chess.Board()
    else:
      self.board = board

  def serialize(self):
    serialized_board = np.zeros(64, np.uint8)
    values = {"P": 1, "N": 2, "B": 3, "R": 4, "Q": 5, "K": 6, \
              "p": 9, "n":10, "b":11, "r":12, "q":13, "k": 14}

    for i in range(64):
      piece = self.board.piece_at(i)
      if piece is not None:
        serialized_board[i] = values[piece.symbol()]
        # TODO: handle special stuff (castling and en passaint)

    return serialized_board

  def edges(self):
    return list(self.board.legal_moves)

  # makes 
  def to_net_input(self):
    s_board = self.serialize()
    # TODO: pass info about the game, such as who's turn it is, etc
    return s_board


In [16]:
data_path = "/content/drive/MyDrive/full_dataset.npz"

data = np.load(data_path)
X, Y = data.f.arr_0, data.f.arr_1

print(X.shape)
print(Y.shape)
print(X)
print(Y)

(1161377, 64)
(1161377,)
[[ 4  2  3 ... 11 10 12]
 [ 4  2  3 ... 11 10 12]
 [ 4  2  3 ... 11 10 12]
 ...
 [ 0  0  0 ... 10  0  0]
 [ 0  0  0 ... 10  0  0]
 [ 0  0  0 ... 10  0  0]]
[1 1 1 ... 1 1 1]


In [ ]:
# TODO: write this
class ValueNet(nn.Module):
  def __init__(self, in_len, out_len):
    super(ValueNet, self).__init__()
    self.fc1 = nn.Linear(in_len, 128)
    self.bn1 = nn.BatchNorm1d(128)
    self.fc2 = nn.Linear(128, 128)
    self.bn2 = nn.Linear(128)
    self.fc3 = nn.Linear(128, out_len)

  def forward(self, x):
    x = F.ReLU(self.bn1(self.fc1(x)))
    x = F.ReLU(self.bn2(self.fc2(x)))
    x = self.fc3(x)
    return x

def predict(model, x):
  return F.sigmoid(model(x))

def save_model(model, path):
  torch.save(model.state_dict(), path)
  print("[+] Model saved at:", path)

def load_model(model, path, val=False):
  model.load_state_dict(torch.load_state_dict(torch.load(path)))
  if val:
    model.eval()
  else:
    model.train()
  print("[+] Loaded model from:", path)

In [ ]:
model_path = "/content/drive/MyDrive/chess_model.pth"

def train(model, X_train, Y_train, X_test=None, Y_test=None):
  lr = 1e-3
  epochs = 100
  BS = 128

  losses, accuracies = [], []

  loss_func = nn.BCELoss()
  optim = torch.optim.Adam(model.parameters(), lr=lr)

  for epoch in epochs:
    print("[+] Epoch", epoch+1)
    for i in (t := trange(0, len(X_train), BS)):
      X = torch.tensor(X_train[i:i+BS]).float().to(device)
      Y = torch.tensor(Y_train[i:i+BS]).float().to(device)

      optim.zero_grad()
      out = model(X)
      loss = loss_func(out, Y).mean()
      loss.backward()
      optim.step()

      # TODO: stats (add tensorboard as well)